In [35]:
from clickhouse_driver import Client
import io
import pandas as pd
import numpy as np
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import psycopg2
import requests
import json
import time
from requests_futures.sessions import FuturesSession
from backend_logic import *
session = FuturesSession()

offerid = '5d03c6207e7680141c2a2cf1'
maxRivalsCount = 12
sqiDiffCoef = 100
maxPos = 10
minCountInSerm = 6

keywords, rivals, startDate, keywordsStr, regionsStr, regions = getRivalsAndData(offerid, maxRivalsCount, sqiDiffCoef, maxPos, minCountInSerm)

In [8]:
all_keywords, notRivals = getKeywordsRivals(keywords, rivals, maxPos, startDate, keywordsStr, regionsStr, regions, minCountInSerm)
all_keywords

[{'searchEngine': 1,
  'isMobile': 0,
  'keyword': 'samura 67 damascus купить',
  'regionId': '213',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 1,
  'isMobile': 0,
  'keyword': 'samura damascus купить',
  'regionId': '213',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'японские кухонные ножи samura',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'купить масленку для сливочного масла',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'сковорода блинная чугунная купить в москве',
  'regionId': '1011969;w%2BCAIQICIUTW9zY293LE1vc2NvdyxSdXNzaWE;Moscow',
  'rivalsCount': 10,
  'regionName': 'Москва'},
 {'searchEngine': 2,
  'isMobile': 0,
  'keyword': 'сковорода из натураль

In [9]:
minKeywordRivals = 2
keywordsToDelete = getKeywordsToRemove(all_keywords, minKeywordRivals, regions)
keywordsToDelete

[{'region': 'Москва',
  'keywordsToDelete': ['бутылка для воды пластиковая многоразовая',
   'umbra в москве',
   'бутылочки для питья']},
 {'region': 'Все регионы',
  'keywordsToDelete': ['Глубокие сковороды москва',
   'joseph Joseph',
   'joseph Joseph купить']}]

In [10]:
filename = 'Rivals ' + offerid + ' ' + 'manual' + '.xlsx'
info = [{
    'Id КП': offerid,
    'Макс. кол-во конкурентов': maxRivalsCount,
    'Макс. отклонение ИКС': sqiDiffCoef,
    'Макс. позиция для поиска конкурентов': maxPos,
    'Мин. кол-во раз, сколько сайт должен встречаться в выдаче, чтобы считаться конкурентом': minCountInSerm,
    'Мин. кол-во конкурентов для запроса': minKeywordRivals
}]
keywordsToDeleteForExcel = []
for item in keywordsToDelete:
    for key in item['keywordsToDelete']:
        keywordsToDeleteForExcel.append({
            'Регион': item['region'],
            'Ключ для удаления': key
        })
writer = pd.ExcelWriter('./downloadable_files/' + filename)
info_df = pd.DataFrame(info)
info_df.to_excel(writer, index=False, sheet_name='Метаданные', freeze_panes = (1,0))
regions_df = pd.DataFrame(regions).rename(columns={'name': 'Регион'})['Регион'].to_frame()
regions_df.to_excel(writer, index=False, sheet_name='Регионы', freeze_panes = (1,0))
rivals_df = pd.DataFrame(rivals).rename(columns={
    'domain': 'Сайт',
    'isExcluded': 'Не конкурент',
    'countInSerm': 'Встречается в выдаче',
})[['Сайт', 'Не конкурент', 'Встречается в выдаче']]
rivals_df.to_excel(writer, index=False, sheet_name='Конкуренты', freeze_panes = (1,0))
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
all_keywords_df.to_excel(writer, index=False, sheet_name='Запросы', freeze_panes = (1,0))
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
keywordsToDeleteForExcel_df.to_excel(writer, index=False, sheet_name='Запросы для удаления', freeze_panes = (1,0))
writer.save()
print(filename)

Rivals 5d03c6207e7680141c2a2cf1 manual.xlsx


In [33]:
keywordsToDeleteForExcel

[{'Регион': 'Москва',
  'Ключ для удаления': 'бутылка для воды пластиковая многоразовая'},
 {'Регион': 'Москва', 'Ключ для удаления': 'umbra в москве'},
 {'Регион': 'Москва', 'Ключ для удаления': 'бутылочки для питья'},
 {'Регион': 'Все регионы', 'Ключ для удаления': 'Глубокие сковороды москва'},
 {'Регион': 'Все регионы', 'Ключ для удаления': 'joseph Joseph'},
 {'Регион': 'Все регионы', 'Ключ для удаления': 'joseph Joseph купить'}]

In [32]:
keywordsToDeleteForExcel_df = pd.DataFrame(keywordsToDeleteForExcel)
all_keywords_df = pd.DataFrame(keywords).rename(columns={
    'keyword': 'Запрос',
    'regionName': 'Регион',
    'searchEngine': 'ПС',
    'isMobile': 'Мобильная выдача',
    'rivalsCount': 'Кол-во конкурентов'
})[['Запрос', 'Кол-во конкурентов', 'Регион', 'ПС', 'Мобильная выдача']]
keywordsToDeleteForExcel_df

,Ключ для удаления,Регион
0,бутылка для воды пластиковая многоразовая,Москва
1,umbra в москве,Москва
2,бутылочки для питья,Москва
3,Глубокие сковороды москва,Все регионы
4,joseph Joseph,Все регионы
5,joseph Joseph купить,Все регионы


In [ ]:
import requests
import json
import pandas as pd

url = 'http://192.168.4.158:8901/api/ExportToExcel'

req = requests.post(url, json.dumps({
    'offerUrl':'5cd19cd07e76802420b8ce98',
    'maxRivalsCount':1000,
    'sqiDiffCoef':100,
    'maxPos':10
}, ensure_ascii=False))

rivals_result = req.json()

In [42]:
from bs4 import BeautifulSoup
domain = 'mac-profi.ru'
r = requests.get('http://' + domain)
encoding = r.encoding if 'charset' in r.headers.get('content-type', '').lower() else None
soup = BeautifulSoup(r.content, from_encoding=encoding)
title = ''
description = ''
icon = 'http://' + domain + '/favicon.ico'
try:
    title = soup.select('title')[0].get_text()
except:
    pass
try:
    description = soup.find("meta", {"name":"description"})['content']
except:
    pass
try:
    icon = soup.find_all(rel = re.compile("Icon|icon"))[0]['href']
    if icon[0] != 'h':
        icon = 'http://' + domain + icon
except:
    pass
res = {
    'title': title,
    'description': description,
    'icon': icon,
    'domain': domain
}
res

{'title': 'Oh!MyGadget! Мак Профи сервисный центр Apple',
 'description': 'Сервисные центры Apple Мак Профи в Москве производят быстрый и качественный ремонт Apple всех моделей! ',
 'icon': 'http://mac-profi.ru/favicon.ico',
 'domain': 'mac-profi.ru'}

In [ ]:
icon = soup.find_all(rel = re.compile("Icon|icon"))
icon

In [5]:
import datetime
mydate = datetime.datetime(2019, 6, 20)
mydate - timedelta(days=1)  

datetime.datetime(2019, 6, 19, 0, 0)